In [ ]:
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/tensorflow_v2/notebooks/3_NeuralNetworks/dcgan.ipynb

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras import Model, layers
from tensorflow.keras.datasets import mnist


In [2]:
gen_lr = 0.002
disc_lr = 0.002

num_features = 784
num_classes = 10

batch_size = 128
noise_dim = 100

In [3]:
( x_train, y_train ), ( x_test, y_test ) = mnist.load_data()

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

x_train, x_test = x_train / 255., x_test / 255.

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(10000).batch(batch_size).prefetch(1)

In [5]:
class Generator(Model):
    # Set layers.
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = layers.Dense(7 * 7 * 128)
        self.bn1 = layers.BatchNormalization()
        self.conv2tr1 = layers.Conv2DTranspose(64, 5, strides=2, padding='SAME')
        self.bn2 = layers.BatchNormalization()
        self.conv2tr2 = layers.Conv2DTranspose(1, 5, strides=2, padding='SAME')

    # Set forward pass.
    def call(self, x, is_training=False):
        x = self.fc1(x)
        x = self.bn1(x, training=is_training)
        x = tf.nn.leaky_relu(x)
        # Reshape to a 4-D array of images: (batch, height, width, channels)
        # New shape: (batch, 7, 7, 128)
        x = tf.reshape(x, shape=[-1, 7, 7, 128])
        # Deconvolution, image shape: (batch, 14, 14, 64)
        x = self.conv2tr1(x)
        x = self.bn2(x, training=is_training)
        x = tf.nn.leaky_relu(x)
        # Deconvolution, image shape: (batch, 28, 28, 1)
        x = self.conv2tr2(x)
        x = tf.nn.tanh(x)
        return x


In [6]:
class Discriminator(Model):

    def __init__(self):
        super(Discriminator, self).__init__()

        self.conv1 = layers.Conv2D(filters=64, kernel_size=5, strides=2, padding='SAME', activation=tf.nn.relu)
        self.bn1 = layers.BatchNormalization()

        self.conv2 = layers.Conv2D(filters=64, kernel_size=2, strides=2, padding='SAME', activation=tf.nn.relu)
        self.bn2 = layers.BatchNormalization()

        self.fc1 = layers.Dense(2)

    def call(self, x, is_training=False):

        x = tf.reshape(x, [-1, 28, 28, 1])

        x = self.conv1(x)
        x = self.bn1(x)

        x = self.conv2(x)
        x = self.bn2(x)

        x = layers.Flatten()(x)

        x = self.fc1(x)

        if not is_training:
            x = tf.nn.softmax(x)
        
        return x

generator = Generator()
discriminator = Discriminator()

In [7]:

def generator_loss(reconstructed_image):
    gen_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=reconstructed_image, labels=tf.ones([batch_size], dtype=tf.int32)))
    return gen_loss

def discriminator_loss(generated_images, real_images):
    fake_loss = tf.reduce_mean( tf.nn.sparse_softmax_cross_entropy_with_logits( logits=generated_images, 
                                labels=tf.zeros([generated_images.shape[0]], dtype=tf.int32) ) )

    real_loss = tf.reduce_mean( tf.nn.sparse_softmax_cross_entropy_with_logits( logits=real_images, 
                                labels=tf.ones([real_images.shape[0]], dtype=tf.int32) ) )

    return fake_loss + real_loss

optimizer_gen = tf.optimizers.Adam(learning_rate=gen_lr)#, beta_1=0.5, beta_2=0.999)
optimizer_disc = tf.optimizers.Adam(learning_rate=disc_lr)#, beta_1=0.5, beta_2=0.999)

In [16]:
def run_optimization(real_images):

    real_images = real_images * 2. - 1.
    noise = np.random.normal(-1., 1., size=[batch_size, noise_dim]).astype(np.float32)


    with tf.GradientTape()  as tape:

        # print(f"Noise shape for generator: {noise.shape}")

        fake_images = generator(noise, is_training=True)

        # print(f"Fake images shape for discriminator: {fake_images.shape}")
        # print(f"Real images shape for discriminator: {real_images.shape}")

        real_pred = discriminator(real_images)
        fake_pred = discriminator(fake_images)

        disc_loss = discriminator_loss(fake_pred, real_pred)


    disc_gradients = tape.gradient(disc_loss, discriminator.trainable_variables)
    optimizer_disc.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))
    # print("Done optimizing for discriminator")

    with tf.GradientTape() as tape:

        # print(f"Noise shape for generator: {noise.shape}")

        generated_images = generator(noise, is_training=True)

        fake_pred = discriminator(generated_images)

        # print(f"Generated images shape by generator: {generated_images.shape}")

        gen_loss = generator_loss(fake_pred)

    # print(generator.trainable_variables)

    gen_gradients = tape.gradient( gen_loss, generator.trainable_variables )
    optimizer_gen.apply_gradients( zip( gen_gradients, generator.trainable_variables ) )
    # print("Done optimizing for generator")

    return gen_loss, disc_loss
    


In [11]:
def run_optimization(real_images):
    
    # Rescale to [-1, 1], the input range of the discriminator
    real_images = real_images * 2. - 1.

    # Generate noise.
    noise = np.random.normal(-1., 1., size=[batch_size, noise_dim]).astype(np.float32)
    
    with tf.GradientTape() as g:
            
        fake_images = generator(noise, is_training=True)
        disc_fake = discriminator(fake_images, is_training=True)
        disc_real = discriminator(real_images, is_training=True)

        disc_loss = discriminator_loss(disc_fake, disc_real)
            
    # Training Variables for each optimizer
    gradients_disc = g.gradient(disc_loss,  discriminator.trainable_variables)
    optimizer_disc.apply_gradients(zip(gradients_disc,  discriminator.trainable_variables))
    
    # Generate noise.
    noise = np.random.normal(-1., 1., size=[batch_size, noise_dim]).astype(np.float32)
    
    with tf.GradientTape() as g:
            
        fake_images = generator(noise, is_training=True)
        disc_fake = discriminator(fake_images, is_training=True)

        gen_loss = generator_loss(disc_fake)
            
    gradients_gen = g.gradient(gen_loss, generator.trainable_variables)
    optimizer_gen.apply_gradients(zip(gradients_gen, generator.trainable_variables))
    
    return gen_loss, disc_loss

In [12]:
training_steps = 20000
display_step = 500

In [17]:
for step, (batch_x, _) in enumerate(train_data.take(training_steps + 1)):
    
    if step == 0:
        # Generate noise.
        noise = np.random.normal(-1., 1., size=[batch_size, noise_dim]).astype(np.float32)

        # print(f"Noise shape in main for loop: {noise.shape}")

        generated_images = generator(noise)

        # print(f"Generated images shape in main for loop: {generated_images.shape}")

        gen_loss = generator_loss(discriminator(generated_images))


        disc_loss = discriminator_loss(discriminator(batch_x), discriminator(generator(noise)))
        # print("initial: gen_loss: %f, disc_loss: %f" % (gen_loss, disc_loss))
        continue
    
    # Run the optimization.
    gen_loss, disc_loss = run_optimization(batch_x)
    
    if step % display_step == 0:
        print("step: %i, gen_loss: %f, disc_loss: %f" % (step, gen_loss, disc_loss))

step: 500, gen_loss: 1.157310, disc_loss: 0.779208
step: 1000, gen_loss: 1.144618, disc_loss: 0.820240
step: 1500, gen_loss: 1.213004, disc_loss: 0.771151
step: 2000, gen_loss: 1.217448, disc_loss: 0.776707
step: 2500, gen_loss: 1.266035, disc_loss: 0.814330
step: 3000, gen_loss: 1.211490, disc_loss: 0.753713
step: 3500, gen_loss: 1.172762, disc_loss: 0.781510
step: 4000, gen_loss: 1.279287, disc_loss: 0.769004
step: 4500, gen_loss: 1.212875, disc_loss: 0.816279
step: 5000, gen_loss: 1.229751, disc_loss: 0.791036
step: 5500, gen_loss: 1.265574, disc_loss: 0.732992
step: 6000, gen_loss: 1.262895, disc_loss: 0.748382
step: 6500, gen_loss: 1.263717, disc_loss: 0.771037
step: 7000, gen_loss: 1.215560, disc_loss: 0.806695
step: 7500, gen_loss: 1.246353, disc_loss: 0.766518
step: 8000, gen_loss: 1.238247, disc_loss: 0.754094
step: 8500, gen_loss: 1.290617, disc_loss: 0.703567
step: 9000, gen_loss: 1.244952, disc_loss: 0.752429
step: 9500, gen_loss: 1.309390, disc_loss: 0.748993
step: 10000, 